## Setup

### Imports

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn import ensemble
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split
import os

## Data

Get PUF records.

In [2]:
# Remove aggregate rows, replace NaN with 0

puf = pd.read_csv('puf2011.csv')

puf = puf[(puf['RECID'] != 999996) &
          (puf['RECID'] != 999997) &
          (puf['RECID'] != 999998) &
          (puf['RECID'] != 999999)
         ]
           
# E19800 and E20100 combined in CPS

puf['E19800_E20100'] = puf['E19800'] + puf['E20100']

# MARS -> category

puf['MARS'] = puf['MARS'].astype('category')

puf['sign'] = np.sign(puf['P22250'])

# Variables shared b/t puf, cps (doesn't include E00650, combines E19800 & E20100)

predictors =  [
              'EIC', 'DSI', 'MARS', 'XTOT', 'E00200', 'E00300', 'E00400', 
              'E00600', 'E00800', 'E00900', 'E01100', 'E01400', 'E01500', 
              'E01700', 'E02100', 'E02300', 'E02400', 'E03150', 'E03210', 
              'E03240', 'E03270', 'E03300', 'E17500', 'E18400', 'E18500', 
              'E19200', 'E19800_E20100','E20400', 'E32800', 'F2441', 'N24'
              ]

keep = ['RECID', 'P22250', 'sign', 'AGIR1'] + predictors

puf = puf[keep]

np.random.seed(1010)

train, test = train_test_split(puf.copy(), test_size=0.2)

In [3]:
X_train = train[predictors]
X_test = test[predictors]

Y_train = train['P22250']
Y_test = test['P22250']

Y_train_sign = train['sign']
Y_test_sign = test['sign']

## Model

Train a random forests model.

In [4]:
# 100 estimators
N_ESTIMATORS = 100
rf = ensemble.RandomForestRegressor(n_estimators=N_ESTIMATORS, 
                                    min_samples_leaf=1, random_state=3, 
                                    verbose=True, 
                                    n_jobs=-1)  # Use maximum number of cores.
rf.fit(X_train, Y_train)

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.0min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=3, verbose=True, warm_start=False)

### Model description

Show the ten most important features.

In [5]:
feature_importance = pd.Series(rf.feature_importances_, index=X_train.columns)
feature_importance.sort_values(ascending=False)[:10]

E00300           0.153667
E00600           0.097812
E19200           0.078997
E03300           0.076603
E03270           0.073783
E00200           0.065784
E18400           0.065574
E00900           0.054671
E19800_E20100    0.050012
E20400           0.047910
dtype: float64

## Predict

### Top-line (average)

In [6]:
pred = pd.DataFrame({'actual': Y_test,
                     'pred': rf.predict(X_test)})
pred['error'] = pred.pred - pred.actual
pred['actual_sign'] = np.sign(pred.actual)
pred['pred_sign'] = np.sign(pred.pred)
pred['correct_sign'] = (pred.actual_sign == pred.pred_sign)
pred['count'] = 1

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.6s finished


MAE, RMSE, and % negative/zero/positive.

In [7]:
pred.error.abs().mean()

42133.70693954704

In [8]:
pred.error.pow(2).mean() ** 0.5

796340.1642941299

In [9]:
pred.pivot_table(index='actual_sign', columns='pred_sign', values='count', 
                 aggfunc=sum, margins=True)

pred_sign,-1.0,0.0,1.0,All
actual_sign,,,,
-1.0,2979,63,1699,4741
0.0,7467,11441,5167,24075
1.0,2391,42,1509,3942
All,12837,11546,8375,32758


In [10]:
pred.correct_sign.mean()

0.4862628976127969

### All trees

Creates array for which rows = puf observations, columns = predictions for each observation from each estimators

In [11]:
preds = []
for estimator in rf.estimators_:
    preds.append(estimator.predict(X_test))
preds = np.array(preds).transpose()  # One row per record.

Picks a random column from the array

In [12]:
rand_col = np.random.randint(N_ESTIMATORS, size=preds.shape[0])
random_tree = preds[np.arange(preds.shape[0]), rand_col]

In [13]:
pred_random_tree = pd.DataFrame({'actual': Y_test,
                                 'pred': random_tree})
pred_random_tree['error'] = pred_random_tree.pred - pred_random_tree.actual
pred_random_tree['actual_sign'] = np.sign(pred_random_tree.actual)
pred_random_tree['pred_sign'] = np.sign(pred_random_tree.pred)
pred_random_tree['correct_sign'] = (
    pred_random_tree.actual_sign == pred_random_tree.pred_sign)
pred_random_tree['count'] = 1

As expected, MAE and RMSE exceed values from the point estimate prediction.

In [14]:
pred_random_tree.error.abs().mean()

57869.098610995185

In [15]:
pred_random_tree.error.pow(2).mean() ** 0.5

872851.0337793779

But the distribution of sign is closer to correct, since it's not averaging out the zeros.

In [16]:
pred_random_tree.pivot_table(index='actual_sign', columns='pred_sign', 
                             values='count', aggfunc=sum, margins=True)

pred_sign,-1.0,0.0,1.0,All
actual_sign,,,,
-1.0,1861,1357,1523,4741
0.0,1610,20987,1478,24075
1.0,1515,1250,1177,3942
All,4986,23594,4178,32758


In [17]:
pred_random_tree.correct_sign.mean()

0.733408633005678

#### Log-loss of sign

In [18]:
preds_neg = np.sum(preds < 0, axis=1) / 100
preds_zero = np.sum(preds == 0, axis=1) / 100
preds_pos = np.sum(preds > 0, axis=1) / 100

rf_pred_proba = list(map(list, zip(*[preds_neg, preds_zero, preds_pos])))

metrics.log_loss(Y_test_sign, rf_pred_proba)
# The result was 0.59 using np.random.seed(100)

0.7124827594128645

## Model

mnlogit

In [21]:
# R-style formula used, otherwise MARS couldn't be encoded as category
# E01100 removed as it causes mnlogit coefficients to be nan

predictors.remove('E01100')
formula = 'sign ~ ' + ' + '.join(predictors)

fit = smf.mnlogit(formula = formula, data = train).fit()
summary = fit.summary()
summary

Optimization terminated successfully.
         Current function value: 0.578467
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                   sign   No. Observations:               131028
Model:                        MNLogit   Df Residuals:                   130962
Method:                           MLE   Df Model:                           64
Date:                Wed, 01 Aug 2018   Pseudo R-squ.:                  0.2352
Time:                        17:23:06   Log-Likelihood:                -75795.
converged:                       True   LL-Null:                       -99104.
                                        LLR p-value:                     0.000
=================================================================================
       sign=0       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         3.0183      0.028    108.201      0.000       2.964       3.073
MARS[T.2]        -0.6085      0.034    -17.763      0.000      -0.676      -0.541
MARS[T.3]        -0.3511      0.069     -5.110      0.000      -0.486      -0.216
MARS[T.4]         0.4550      0.061      7.474      0.000       0.336       0.574
EIC               1.2993      0.073     17.741      0.000       1.156       1.443
DSI               0.9788      0.110      8.917      0.000       0.764       1.194
XTOT             -0.1144      0.017     -6.782      0.000      -0.147      -0.081
E00200        -7.765e-07   2.25e-08    -34.577      0.000   -8.21e-07   -7.33e-07
E00300        -2.655e-06   1.74e-07    -15.267      0.000      -3e-06   -2.31e-06
E00400        -3.212e-06   2.22e-07    -14.470      0.000   -3.65e-06   -2.78e-06
E00600        -1.316e-06   1.04e-07    -12.638      0.000   -1.52e-06   -1.11e-06
E00800        -1.891e-05   5.35e-06     -3.532      0.000   -2.94e-05   -8.42e-06
E00900        -1.282e-09   3.78e-08     -0.034      0.973   -7.54e-08    7.28e-08
E01400        -1.575e-06   1.45e-07    -10.896      0.000   -1.86e-06   -1.29e-06
E01500        -3.761e-07   2.97e-08    -12.644      0.000   -4.34e-07   -3.18e-07
E01700         -4.23e-07   2.14e-07     -1.977      0.048   -8.42e-07    -3.7e-09
E02100         3.694e-07   1.44e-07      2.573      0.010     8.8e-08    6.51e-07
E02300         2.983e-05   4.47e-06      6.678      0.000    2.11e-05    3.86e-05
E02400        -1.541e-05   8.63e-07    -17.844      0.000   -1.71e-05   -1.37e-05
E03150        -4.883e-05   6.21e-06     -7.863      0.000    -6.1e-05   -3.67e-05
E03210            0.0007   6.78e-05     10.766      0.000       0.001       0.001
E03240         4.695e-07   2.99e-07      1.569      0.117   -1.17e-07    1.06e-06
E03270        -2.243e-05   1.92e-06    -11.656      0.000   -2.62e-05   -1.87e-05
E03300        -1.196e-05   6.91e-07    -17.303      0.000   -1.33e-05   -1.06e-05
E17500        -6.538e-06   1.03e-06     -6.369      0.000   -8.55e-06   -4.53e-06
E18400         -1.25e-06   1.37e-07     -9.156      0.000   -1.52e-06   -9.82e-07
E18500        -3.321e-05      1e-06    -33.048      0.000   -3.52e-05   -3.12e-05
E19200         -7.01e-07   3.76e-07     -1.862      0.063   -1.44e-06    3.69e-08
E19800_E20100 -3.258e-07   1.17e-07     -2.776      0.005   -5.56e-07   -9.58e-08
E20400        -4.446e-05   1.13e-06    -39.397      0.000   -4.67e-05   -4.22e-05
E32800            0.0001   2.35e-05      4.781      0.000    6.63e-05       0.000
F2441            -0.1526      0.050     -3.059      0.002      -0.250      -0.055
N24               0.1175      0.019      6.285      0.000       0.081       0.154
---------------------------------------------------------------------------------
       sign=1       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.1370      0.034     -4.

Mnlogit.predict() strips categorical outcomes of their labels. Common sense dictates that column 1 = P(0/zero), and we know from documentation that column 0 = P(ommitted category), which is P(-1/negative) based on the summary. Finally by process of elimination, column 2 = P(1/positive)

I have opened an issue in statsmodels github here: https://github.com/statsmodels/statsmodels/issues/4804

In [22]:
pred1 = fit.predict(X_test)
pred1.head()

,0,1,2
160193,0.127306,0.761826,0.110868
68268,0.429558,0.262402,0.308040
59364,0.093445,0.826134,0.080421
161835,0.096010,0.820993,0.082996
101299,0.046216,0.910594,0.043190


Renaming columns to reflect sign categories.

In [23]:
pred1.rename(columns = {0: -1, 1: 0, 2: 1}, inplace = True)
pred1.head()

,-1,0,1
160193,0.127306,0.761826,0.110868
68268,0.429558,0.262402,0.308040
59364,0.093445,0.826134,0.080421
161835,0.096010,0.820993,0.082996
101299,0.046216,0.910594,0.043190


### Stochastic prediction function

In [24]:
# Arguments = 'runif': a random uniform in [0,1], and 'row': a vector of predicted probabilities for each sign.
# Output = predicted sign if associated CDF value > runif.
# probs[0] = P(neg), probs[1] = P(zero), probs[2] = P(pos)

def stoch_imp(runif, row):
    
    if runif < row[-1]:
        return (-1.)
    
    elif runif < row[-1] + row[0]:
        return (0.)
    
    else:
        return (1.)

# Vector of runifs the length of test

np.random.seed(1995)
runifs = np.random.uniform(size = len(test))

Loop through pred1, outputting stoch_imp(runif, row)

In [25]:
mnl_pred_proba = []

count = 0

for i in runifs:
    probs = pred1.iloc[count]
    mnl_pred_proba.append(stoch_imp(i, probs))
    count += 1

% accuracy

In [26]:
(mnl_pred_proba == Y_test_sign).mean()

0.6790097075523536

#### Log-loss of sign

In [27]:
metrics.log_loss(Y_test_sign, pred1)

0.5787191530726037